# Query Model Repository for best anomaly detection model

In [4]:
from hops import model
from hops.model import Metric
MODEL_NAME="ganAml"
EVALUATION_METRIC="loss"

In [5]:
best_model = model.get_best_model(MODEL_NAME, EVALUATION_METRIC, Metric.MIN)

In [6]:
print('Model name: ' + best_model['name'])
print('Model version: ' + str(best_model['version']))
print(best_model['metrics'])

Model name: ganAml
Model version: 1
{'loss': '-0.49080216884613037'}

# Create Model Serving of Exported Model

In [7]:
from hops import serving

In [8]:
MODEL_NAME

'ganAml'

In [9]:
# Create serving
model_path="/Models/" + best_model['name']
model_path
response = serving.create_or_update(artifact_path=model_path, serving_name=MODEL_NAME, serving_type="TENSORFLOW", 
                                 model_version=best_model['version'])

Creating a serving for model ganAml ...
Serving for model ganAml successfully created

In [10]:
# List all available servings in the project
for s in serving.get_all():
    print(s.name)

ganAml

In [11]:
# Get serving status
serving.get_status(MODEL_NAME)

'Running'

# Start Model Serving Server

In [12]:
if serving.get_status(MODEL_NAME) == 'Stopped':
    serving.start(MODEL_NAME)

In [13]:
import time
while serving.get_status(MODEL_NAME) != "Running":
    time.sleep(5) # Let the serving startup correctly
time.sleep(5)

# Send Prediction Requests to the Served Model using Hopsworks REST API

In [14]:
import hsfs
# Create a connection
connection = hsfs.connection()
# Get the feature store handle for the project's feature store
fs = connection.get_feature_store()

Connected. Call `.close()` to terminate connection gracefully.

In [15]:
eval_td = fs.get_training_dataset("gan_eval_df", 1)

In [16]:
import numpy as np
eval_td.read().select("embedding").rdd.map(lambda x: np.array(x)[0].tolist()).first()

[-0.5829833149909973, -0.9786504507064819, -0.13974636793136597, -0.08100467175245285, -0.7230328321456909, -0.6167192459106445, 0.3849627375602722, -0.8253325819969177, -0.6468784809112549, 0.09811028093099594, 0.9364645481109619, -0.7005855441093445, 0.48287415504455566, 1.1603587865829468]

In [74]:
TOPIC_NAME = serving.get_kafka_topic(MODEL_NAME)
import json

input = [0.37102198600769043, -0.09608001261949539, -0.9585465788841248, 0.08975595980882645, 0.4234561026096344, -0.34848344326019287, 1.2133934497833252, 0.3718363344669342, -0.7127293944358826, -0.5090847611427307, 0.5293884873390198, -0.18760253489017487, 0.9332895874977112, -0.5815020203590393]
data = {"signature_name": "serving_default", "input": [input]}
response = serving.make_inference_request(MODEL_NAME, data)
print(response)

An error was encountered:
Could not create or update serving (url: /hopsworks-api/api/project/119/inference/models/ganAml:predict), server response: 
 HTTP code: 400, HTTP reason: Bad Request, error code: 250008, error msg: Serving instance bad request error, user msg: {
    "error": "Serving signature name: \"serving_default\" not found in signature def"
}
Traceback (most recent call last):
  File "/srv/hops/anaconda/envs/theenv/lib/python3.7/site-packages/hops/serving.py", line 527, in make_inference_request
    return _make_inference_request_rest(serving_name, data, verb)
  File "/srv/hops/anaconda/envs/theenv/lib/python3.7/site-packages/hops/serving.py", line 562, in _make_inference_request_rest
    error_code, error_msg, user_msg))
hops.exceptions.RestAPIError: Could not create or update serving (url: /hopsworks-api/api/project/119/inference/models/ganAml:predict), server response: 
 HTTP code: 400, HTTP reason: Bad Request, error code: 250008, error msg: Serving instance bad requ

In [1]:
import tensorflow as tf
import pydoop.hdfs as pydoop

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log
51,application_1607949680860_0053,pyspark,idle,Link,Link


SparkSession available as 'spark'.


In [2]:
loaded = tf.saved_model.load(pydoop.path.abspath("hdfs:///Projects/amlsim/Models/ganAml/1"))

In [20]:
print(list(loaded.signatures))

['serving_default']

In [64]:
infer = loaded.signatures["serving_default"]

In [65]:
infer

<ConcreteFunction signature_wrapper(input) at 0x7F9A3992F7D0>

In [24]:
print(infer.structured_outputs)

{'anomaly_score': TensorSpec(shape=(None,), dtype=tf.float32, name='anomaly_score')}

In [55]:
from tensorflow.python.tools import saved_model_utils
saved_model_dir = 'hdfs:///Projects/amlsim/Models/ganAml/1'
tag_sets = saved_model_utils.get_saved_model_tag_sets(saved_model_dir)
for tag_set in sorted(tag_sets):
    print("\nMetaGraphDef with tag-set: '%s' "
          "contains the following SignatureDefs:" % ', '.join(tag_set))

    tag_set = ','.join(tag_set)
meta_graph = saved_model_utils.get_meta_graph_def(saved_model_dir, tag_set)
meta_graph.signature_def    


MetaGraphDef with tag-set: 'serve' contains the following SignatureDefs:
{'__saved_model_init_op': outputs {
  key: "__saved_model_init_op"
  value {
    name: "NoOp"
    tensor_shape {
      unknown_rank: true
    }
  }
}
, 'serving_default': inputs {
  key: "input"
  value {
    name: "serving_default_input:0"
    dtype: DT_FLOAT
    tensor_shape {
      dim {
        size: -1
      }
      dim {
        size: -1
      }
    }
  }
}
outputs {
  key: "anomaly_score"
  value {
    name: "StatefulPartitionedCall:0"
    dtype: DT_FLOAT
    tensor_shape {
      dim {
        size: -1
      }
    }
  }
}
method_name: "tensorflow/serving/predict"
}

In [77]:
#########################################################
# https://github.com/tensorflow/tensorflow/issues/31421
# https://github.com/Lornatang/TensorFlow2-tutorials/blob/master/guide/serialization/saved_model.py

In [78]:
from tensorflow.python.tools import saved_model_utils
saved_model_dir = 'hdfs:///Projects/amlsim/Models/NodeEmbeddings/1'
tag_sets = saved_model_utils.get_saved_model_tag_sets(saved_model_dir)
for tag_set in sorted(tag_sets):
    print("\nMetaGraphDef with tag-set: '%s' "
          "contains the following SignatureDefs:" % ', '.join(tag_set))

    tag_set = ','.join(tag_set)
meta_graph = saved_model_utils.get_meta_graph_def(saved_model_dir, tag_set)
meta_graph.signature_def  


MetaGraphDef with tag-set: 'serve' contains the following SignatureDefs:
{'__saved_model_init_op': outputs {
  key: "__saved_model_init_op"
  value {
    name: "NoOp"
    tensor_shape {
      unknown_rank: true
    }
  }
}
, 'serving_default': inputs {
  key: "input_1"
  value {
    name: "serving_default_input_1:0"
    dtype: DT_FLOAT
    tensor_shape {
      dim {
        size: -1
      }
      dim {
        size: 1
      }
    }
  }
}
inputs {
  key: "input_2"
  value {
    name: "serving_default_input_2:0"
    dtype: DT_FLOAT
    tensor_shape {
      dim {
        size: -1
      }
      dim {
        size: 1
      }
    }
  }
}
outputs {
  key: "reshape_2"
  value {
    name: "StatefulPartitionedCall:0"
    dtype: DT_FLOAT
    tensor_shape {
      dim {
        size: -1
      }
      dim {
        size: 1
      }
    }
  }
}
method_name: "tensorflow/serving/predict"
}